In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import string

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/korea-university-cose362-2020-2/train.csv
/kaggle/input/korea-university-cose362-2020-2/leaderboard_test_file.csv


In [2]:
train = pd.read_csv('../input/korea-university-cose362-2020-2/train.csv') #training set

train.head()

,id,label,message
0,0,0,Hey tmr maybe can meet you at yck
1,1,0,Smith waste da.i wanna gayle.
2,2,1,Well done ENGLAND! Get the official poly ringt...
3,3,0,Just finished. Missing you plenty
4,4,0,U too...


if the message is spam, label is 1.
otherwise, 0.

In [3]:
test = pd.read_csv('../input/korea-university-cose362-2020-2/leaderboard_test_file.csv') #test set

test.head()

,id,message
0,0,"SPJanuary Male Sale! Hot Gay chat now cheaper,..."
1,1,have * good weekend.
2,2,Hope this text meets you smiling. If not then ...
3,3,Please tell me you have some of that special s...
4,4,He said i look pretty wif long hair wat. But i...


**get clean words data**

In [24]:
#removing punctuations, tokenizing function

import nltk
from nltk.corpus import stopwords

def getCleanWords(text):
    
    #remove punctuations
    no_punc = [char for char in text if char not in string.punctuation]
    no_punc = ''.join(no_punc)
    
    #tokenizing
    clean_strings = no_punc.split(' ')
    
    #no stop words
    stop_words = set(stopwords.words('english'))
    clean_words = [w for w in clean_strings if not w in stop_words] 
    
    return clean_words

In [25]:
train['message'].head().apply(getCleanWords)

0                         [Hey, tmr, maybe, meet, yck]
1                    [Smith, waste, dai, wanna, gayle]
2    [Well, done, ENGLAND, Get, official, poly, rin...
3                    [Just, finished, Missing, plenty]
4                                                  [U]
Name: message, dtype: object

In [26]:
test['message'].head().apply(getCleanWords)

0    [SPJanuary, Male, Sale, Hot, Gay, chat, cheape...
1                                    [, good, weekend]
2    [Hope, text, meets, smiling, If, let, text, gi...
3              [Please, tell, special, stock, talking]
4    [He, said, look, pretty, wif, long, hair, wat,...
Name: message, dtype: object

**Classification**

In [39]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(analyzer=getCleanWords)

train_data = vectorizer.fit_transform(train['message'])

train_data.shape

(4000, 9843)

In [40]:
test_data = vectorizer.transform(test['message'])

test_data.shape

(300, 9843)

In [33]:
from sklearn.model_selection import train_test_split

X_train = train_data
y_train = train['label']

X_test = test_data

In [41]:
from sklearn.naive_bayes import MultinomialNB

classifier = MultinomialNB()

classifier.fit(X_train, y_train)

MultinomialNB()

In [42]:
#Print the predictions
print(classifier.predict(X_train))

#Print the actual values
print(y_train.values)

[0 0 1 ... 0 0 0]
[0 0 1 ... 0 0 0]


In [43]:
#Evaluate the model on the training data set
from sklearn.metrics import classification_report,confusion_matrix, accuracy_score

pred = classifier.predict(X_train)
print(classification_report(y_train ,pred ))
print('Confusion Matrix: \n',confusion_matrix(y_train,pred))
print()
print('Accuracy: ', accuracy_score(y_train,pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3500
           1       0.99      0.98      0.98       500

    accuracy                           1.00      4000
   macro avg       0.99      0.99      0.99      4000
weighted avg       1.00      1.00      1.00      4000

Confusion Matrix: 
 [[3494    6]
 [  11  489]]

Accuracy:  0.99575


In [45]:
#Print the predictions

pred_test = classifier.predict(X_test)
print('Predicted value: ',pred_test)

Predicted value:  [1 0 0 0 0 0 0 0 0 0 1 0 1 1 0 0 1 1 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 1 1 1 0 0 0 0 0 1 0 1 0 0 0
 0 1 0 1 0 0 1 1 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 1 1 1 0 0 0 1 0 0 1 0 0 0 0
 0 1 1 1 1 0 0 1 1 1 0 0 0 0 1 0 0 1 1 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 1 0 0 0 0 1 1 0 1 1 1 1 0 0 1 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 1 0 1
 0 0 1 0 1 1 1 1 0 0 0 0 0 0 1 0 0 1 0 0 0 0 1 0 0 1 0 0 0 0 0 1 0 0 1 0 0
 1 0 0 0 1 0 0 0 0 0 1 1 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 1 1 0 0 0 0 0
 0 0 0 1]


**Result**

In [55]:
result = test.drop(columns=['message'])

result['label'] = pred_test

result.head()

,id,label
0,0,1
1,1,0
2,2,0
3,3,0
4,4,0


In [59]:
result.to_csv('./spamLabelResult.csv', index = False)

leaderboard result: 0.89570